In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium

In [2]:
from update_vars import analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH

In [3]:
AM_PEAK

(datetime.time(6, 0), datetime.time(9, 0))

In [4]:
PM_PEAK

(datetime.time(15, 0), datetime.time(19, 0))

In [5]:
imported_st = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
)

In [6]:
am_peak_hrs = list(range(AM_PEAK[0].hour, AM_PEAK[1].hour))
pm_peak_hrs = list(range(PM_PEAK[0].hour, PM_PEAK[1].hour))
both_peaks_hrs = am_peak_hrs + pm_peak_hrs
peaks_dict = {key: 'am_peak' for key in am_peak_hrs} | {key: 'pm_peak' for key in pm_peak_hrs}

#  happens in st agg...
imported_st = imported_st.assign(
        departure_hour = pd.to_datetime(
            imported_st.departure_sec, unit="s").dt.hour
    )

both_peak_filtered = imported_st[imported_st['arrival_hour'].isin(both_peaks_hrs)]
both_peak_filtered['peak'] = both_peak_filtered['arrival_hour'].map(peaks_dict)

/tmp/ipykernel_639/1642475809.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_peak_filtered['peak'] = both_peak_filtered['arrival_hour'].map(peaks_dict)


In [7]:
peaks_dict

{6: 'am_peak',
 7: 'am_peak',
 8: 'am_peak',
 15: 'pm_peak',
 16: 'pm_peak',
 17: 'pm_peak',
 18: 'pm_peak'}

## sjoin stops to segs -- subset peak?

In [8]:
import sjoin_stops_to_segments

In [9]:
imported_st.dtypes

feed_key           object
feed_timezone      object
base64_url         object
trip_id            object
stop_id            object
stop_sequence       int64
timepoint         float64
arrival_sec       float64
departure_sec     float64
arrival_hour      float64
departure_hour    float64
dtype: object

In [10]:
imported_st >> head(3)

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour
0,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,dgxbvk5qu,11,1,1.0,43200.0,43200.0,12.0,12.0
1,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,992,GA,1,1.0,85200.0,85200.0,23.0,23.0
2,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,ogkycex13,11,1,1.0,15000.0,15000.0,4.0,4.0


In [11]:
both_peak_filtered >> head(2)

,feed_key,feed_timezone,base64_url,trip_id,stop_id,stop_sequence,timepoint,arrival_sec,departure_sec,arrival_hour,departure_hour,peak
3,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,hcm9spuol,11,1,1.0,59400.0,59400.0,16.0,16.0,pm_peak
5,55d8763ca8845a4efa50e40a9b972e1c,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,1335,LTP,1,1.0,64080.0,64080.0,17.0,17.0,pm_peak


In [13]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date)

In [ ]:
# test = dev_stop_times_aggregation_max_by_stop(both_peak_filtered, analysis_date, single_route_dir=False)

# old_hq = max_arrivals_by_stop >> filter(_.am_max_trips > 4, _.pm_max_trips > 4) # old HQ corridor + major stop precursor
# old_hq

# new_hq_multi = test >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (still multi-route)
# new_hq_multi

# new_ms_multi = test >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (still multi-route)
# new_ms_multi

# single_route_test = dev_stop_times_aggregation_max_by_stop(both_peak_filtered, analysis_date, single_route_dir=True)

# new_hq_single = single_route_test >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (single-route)
# new_hq_single

# new_ms_single = single_route_test >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (single-route)
# new_ms_single

## preliminary mapping

* add bus/rail/ferry MTS in orange to help decide

In [ ]:
EXPORT_PATH

In [ ]:
hq_points = gpd.read_parquet(f'{EXPORT_PATH}ca_hq_transit_stops.parquet')

In [ ]:
non_bus = hq_points >> filter(-_.hqta_type.isin(['hq_corridor_bus', 'major_stop_bus']))

In [ ]:
both_peak_filtered

In [ ]:
gtfs_utils_v2.get_stops?

In [ ]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=both_peak_filtered.feed_key.unique(),
                       stop_cols=['feed_key', 'stop_id'])

In [ ]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date) >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key)

In [ ]:
stops = stops >> inner_join(_, feeds, on='feed_key')

In [ ]:
def add_to_map(test_df, m=None, **kwargs):
    gdf = stops >> inner_join(_, test_df, on=['schedule_gtfs_dataset_key', 'stop_id'])
    m = gdf.explore(m = m, **kwargs)
    return m

In [ ]:
m = add_to_map(old_hq)
m = non_bus.explore(m = m, color='orange')

In [ ]:
m = add_to_map(new_hq_multi, m=m, color='green')

In [ ]:
m = add_to_map(new_hq_single, m=m, color='red')

In [ ]:
folium.LayerControl().add_to(m)

In [ ]:
m

## major stops

In [ ]:
m = add_to_map(old_hq)
m = non_bus.explore(m = m, color='orange')

In [ ]:
m = add_to_map(new_ms_multi, m=m, color='green')

In [ ]:
m = add_to_map(new_ms_single, m=m, color='red')

In [ ]:
folium.LayerControl().add_to(m)

In [ ]:
m